In [1]:
import pandas as pd
import math

# Сырые данные
Читаем лист из эксельки, в качестве заголовков – первые 4 строки

In [2]:
N_ATOMS = 3
EXCEL_PATH = "All Molecules For Scale Factors.xlsx"

In [3]:
raw_data = pd.read_excel(
    EXCEL_PATH,
    sheet_name=f"{N_ATOMS}atoms",
    header=(0, 1, 2, 3),
)
raw_data.head()

Unnamed: 0_level_0   Unnamed: 1_level_0 Unnamed: 2_level_0  \
  Unnamed: 0_level_1   Unnamed: 1_level_1 Unnamed: 2_level_1   
  Unnamed: 0_level_2   Unnamed: 1_level_2 Unnamed: 2_level_2   
                   #             Molecule             Brutto   
0                1.0     Hydrogen cyanide                HCN   
1                NaN                  NaN                NaN   
2                NaN                  NaN                NaN   
3                NaN                  NaN                NaN   
4                2.0  hydrogen isocyanide                HNC   

  Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0     Method:  \
  Unnamed: 3_level_1 Unnamed: 4_level_1 Unnamed: 5_level_1 Dispersion:   
  Unnamed: 3_level_2 Unnamed: 4_level_2 Unnamed: 5_level_2      Basis:   
  Fundamental [cm-1]             Source     Who calculated    Property   
0              712.0                NaN          Nikolenko         NaN   
1             2089.0                NaN                NaN         NaN   
2             3312.0                NaN                NaN         NaN   
3                NaN                NaN                NaN         NaN   
4              477.0                NaN          Nikolenko         NaN   

      BLYP                      ...     PBE0                                  \
    D3(BJ)                      ...       D4                                   
     6-31G            def2-SVP  ... def2-SVP            def2-TZVP              
  ν [cm-1] Exp./Calc. ν [cm-1]  ... ν [cm-1] Exp./Calc.  ν [cm-1] Exp./Calc.   
0      NaN        NaN      NaN  ...      NaN        NaN       NaN        NaN   
1      NaN        NaN      NaN  ...      NaN        NaN       NaN        NaN   
2      NaN        NaN      NaN  ...      NaN        NaN       NaN        NaN   
3      NaN        NaN      NaN  ...      NaN        NaN       NaN        NaN   
4      NaN        NaN      NaN  ...      NaN        NaN       NaN        NaN   

                HF-3c                                 PBEh-3c  \
  Unnamed: 55_level_1 Unnamed: 56_level_1 Unnamed: 57_level_1   
  Unnamed: 55_level_2 Unnamed: 56_level_2 Unnamed: 57_level_2   
             ν [cm-1]          Exp./Calc.            ν [cm-1]   
0                 NaN                 NaN                 NaN   
1                 NaN                 NaN                 NaN   
2                 NaN                 NaN                 NaN   
3                 NaN                 NaN                 NaN   
4                 NaN                 NaN                 NaN   

                                r2SCAN-3c                      
  Unnamed: 58_level_1 Unnamed: 59_level_1 Unnamed: 60_level_1  
  Unnamed: 58_level_2 Unnamed: 59_level_2 Unnamed: 60_level_2  
           Exp./Calc.            ν [cm-1]          Exp./Calc.  
0                 NaN                 NaN                 NaN  
1                 NaN                 NaN                 NaN  
2                 NaN                 NaN                 NaN  
3                 NaN                 NaN                 NaN  
4                 NaN                 NaN                 NaN  

[5 rows x 61 columns]

Делаем вменяемый горизонтальный индекс

In [4]:
# Здесь хочется избавиться от всех полей 'Unnamed: ...'
raw_data.columns[0]

('Unnamed: 0_level_0', 'Unnamed: 0_level_1', 'Unnamed: 0_level_2', '#')

In [5]:
# Это можно просто соединить, в целом в этой колонке и данных-то быть не должно
raw_data.columns[6]

('Method:', 'Dispersion:', 'Basis:', 'Property')

In [6]:
# поэтому заменим их на количество атомов
raw_data.drop(raw_data.columns[6], axis=1, inplace=True)
raw_data[("", "", "", "N atoms")] = N_ATOMS
raw_data.head(5)

Unnamed: 0_level_0   Unnamed: 1_level_0 Unnamed: 2_level_0  \
  Unnamed: 0_level_1   Unnamed: 1_level_1 Unnamed: 2_level_1   
  Unnamed: 0_level_2   Unnamed: 1_level_2 Unnamed: 2_level_2   
                   #             Molecule             Brutto   
0                1.0     Hydrogen cyanide                HCN   
1                NaN                  NaN                NaN   
2                NaN                  NaN                NaN   
3                NaN                  NaN                NaN   
4                2.0  hydrogen isocyanide                HNC   

  Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0     BLYP  \
  Unnamed: 3_level_1 Unnamed: 4_level_1 Unnamed: 5_level_1   D3(BJ)   
  Unnamed: 3_level_2 Unnamed: 4_level_2 Unnamed: 5_level_2    6-31G   
  Fundamental [cm-1]             Source     Who calculated ν [cm-1]   
0              712.0                NaN          Nikolenko      NaN   
1             2089.0                NaN                NaN      NaN   
2             3312.0                NaN                NaN      NaN   
3                NaN                NaN                NaN      NaN   
4              477.0                NaN          Nikolenko      NaN   

                                  ...       PBE0                       \
                                  ...         D4                        
             def2-SVP             ...   def2-SVP def2-TZVP              
  Exp./Calc. ν [cm-1] Exp./Calc.  ... Exp./Calc.  ν [cm-1] Exp./Calc.   
0        NaN      NaN        NaN  ...        NaN       NaN        NaN   
1        NaN      NaN        NaN  ...        NaN       NaN        NaN   
2        NaN      NaN        NaN  ...        NaN       NaN        NaN   
3        NaN      NaN        NaN  ...        NaN       NaN        NaN   
4        NaN      NaN        NaN  ...        NaN       NaN        NaN   

                HF-3c                                 PBEh-3c  \
  Unnamed: 55_level_1 Unnamed: 56_level_1 Unnamed: 57_level_1   
  Unnamed: 55_level_2 Unnamed: 56_level_2 Unnamed: 57_level_2   
             ν [cm-1]          Exp./Calc.            ν [cm-1]   
0                 NaN                 NaN                 NaN   
1                 NaN                 NaN                 NaN   
2                 NaN                 NaN                 NaN   
3                 NaN                 NaN                 NaN   
4                 NaN                 NaN                 NaN   

                                r2SCAN-3c                              
  Unnamed: 58_level_1 Unnamed: 59_level_1 Unnamed: 60_level_1          
  Unnamed: 58_level_2 Unnamed: 59_level_2 Unnamed: 60_level_2          
           Exp./Calc.            ν [cm-1]          Exp./Calc. N atoms  
0                 NaN                 NaN                 NaN       3  
1                 NaN                 NaN                 NaN       3  
2                 NaN                 NaN                 NaN       3  
3                 NaN                 NaN                 NaN       3  
4                 NaN                 NaN                 NaN       3  

[5 rows x 61 columns]

In [7]:
# На случай, если оставляем мультииндекс 
def clean_multiindex(multiindex: tuple) -> tuple:
    return tuple(
        index if not index.startswith("Unnamed: ") else ""
        for index in multiindex
    )
    
# Если делаем соединялку
def merge_multiindex(multiindex: tuple) -> str:
    return "_".join(
        index.strip(":")
        for index in multiindex
        if not index.startswith("Unnamed: ")
    )

In [8]:
cleaned_multiindex = pd.MultiIndex.from_tuples(
    [
        (clean_multiindex(idx))
        for idx in raw_data.columns
    ],
    names=('Method', 'Dispersion', 'Basis', 'Property')
)

In [9]:
cleaned_multiindex

MultiIndex([(         '',       '',          '',                  '#'),
            (         '',       '',          '',           'Molecule'),
            (         '',       '',          '',             'Brutto'),
            (         '',       '',          '', 'Fundamental [cm-1]'),
            (         '',       '',          '',             'Source'),
            (         '',       '',          '',     'Who calculated'),
            (     'BLYP', 'D3(BJ)',     '6-31G',           'ν [cm-1]'),
            (     'BLYP', 'D3(BJ)',     '6-31G',         'Exp./Calc.'),
            (     'BLYP', 'D3(BJ)',  'def2-SVP',           'ν [cm-1]'),
            (     'BLYP', 'D3(BJ)',  'def2-SVP',         'Exp./Calc.'),
            (     'BLYP', 'D3(BJ)', 'def2-TZVP',           'ν [cm-1]'),
            (     'BLYP', 'D3(BJ)', 'def2-TZVP',         'Exp./Calc.'),
            (     'BLYP',     'D4',     '6-31G',           'ν [cm-1]'),
            (     'BLYP',     'D4',     '6-31G',         'Exp./C

In [10]:
# Чистим названия колонок
flat_index_data = pd.DataFrame(columns=cleaned_multiindex)
for src_col, dst_col in zip(raw_data.columns, flat_index_data.columns):
    flat_index_data[dst_col] = raw_data[src_col]
flat_index_data.head()

Method                                                                 \
Dispersion                                                              
Basis                                                                   
Property      #             Molecule Brutto Fundamental [cm-1] Source   
0           1.0     Hydrogen cyanide    HCN              712.0    NaN   
1           NaN                  NaN    NaN             2089.0    NaN   
2           NaN                  NaN    NaN             3312.0    NaN   
3           NaN                  NaN    NaN                NaN    NaN   
4           2.0  hydrogen isocyanide    HNC              477.0    NaN   

Method                        BLYP                                 ...  \
Dispersion                  D3(BJ)                                 ...   
Basis                        6-31G            def2-SVP             ...   
Property   Who calculated ν [cm-1] Exp./Calc. ν [cm-1] Exp./Calc.  ...   
0               Nikolenko      NaN        NaN      NaN        NaN  ...   
1                     NaN      NaN        NaN      NaN        NaN  ...   
2                     NaN      NaN        NaN      NaN        NaN  ...   
3                     NaN      NaN        NaN      NaN        NaN  ...   
4               Nikolenko      NaN        NaN      NaN        NaN  ...   

Method           PBE0                         HF-3c             PBEh-3c  \
Dispersion         D4                                                     
Basis        def2-SVP def2-TZVP                                           
Property   Exp./Calc.  ν [cm-1] Exp./Calc. ν [cm-1] Exp./Calc. ν [cm-1]   
0                 NaN       NaN        NaN      NaN        NaN      NaN   
1                 NaN       NaN        NaN      NaN        NaN      NaN   
2                 NaN       NaN        NaN      NaN        NaN      NaN   
3                 NaN       NaN        NaN      NaN        NaN      NaN   
4                 NaN       NaN        NaN      NaN        NaN      NaN   

Method                r2SCAN-3c                     
Dispersion                                          
Basis                                               
Property   Exp./Calc.  ν [cm-1] Exp./Calc. N atoms  
0                 NaN       NaN        NaN       3  
1                 NaN       NaN        NaN       3  
2                 NaN       NaN        NaN       3  
3                 NaN       NaN        NaN       3  
4                 NaN       NaN        NaN       3  

[5 rows x 61 columns]

# Аннотируем все данные заголовками слева
Сейчас таблица выглядит так:

|    |   # | Molecule            | Brutto   |   Fundamental [cm-1] |   Source | Who calculated   |   Method_Dispersion_Basis_Property |   BLYP_D3(BJ)_6-31G_ν [cm-1] |
|---:|----:|:--------------------|:---------|---------------------:|---------:|:-----------------|-----------------------------------:|-----------------------------:|
|  0 |   1 | Hydrogen cyanide    | HCN      |                  712 |      nan | Nikolenko        |                                nan |                          nan |
|  1 | nan | nan                 | nan      |                 2089 |      nan | nan              |                                nan |                          nan |
|  2 | nan | nan                 | nan      |                 3312 |      nan | nan              |                                nan |                          nan |
|  3 | nan | nan                 | nan      |                  nan |      nan | nan              |                                nan |                          nan |
|  4 |   2 | hydrogen isocyanide | HNC      |                  477 |      nan | Nikolenko        |                                nan |                          nan |

Но удобнее, когда для каждой частоты можно узнать молекулу, не зная контекста
|    |   # | Molecule            | Brutto   |   Fundamental [cm-1] |   Source | Who calculated   |   Method_Dispersion_Basis_Property |   BLYP_D3(BJ)_6-31G_ν [cm-1] |
|---:|----:|:--------------------|:---------|---------------------:|---------:|:-----------------|-----------------------------------:|-----------------------------:|
|  0 |   1 | Hydrogen cyanide    | HCN      |                  712 |      nan | Nikolenko        |                                nan |                          nan |
|  1 | 1 | Hydrogen cyanide     | HCN      |                 2089 |      nan | Nikolenko              |                                nan |                          nan |
|  2 | 1 | Hydrogen cyanide                 | HCN      |                 3312 |      nan | Nikolenko              |                                nan |                          nan |
|  3 | 1 | Hydrogen cyanide                 | HCN      |                  nan |      nan | Nikolenko              |                                nan |                          nan |
|  4 |   2 | hydrogen isocyanide | HNC      |                  477 |      nan | Nikolenko        |                                nan |                          nan |

In [11]:
def assign_missing_properties(df):
    # current context
    mol_no, mol_name, brutto, owner = [None] * 4
    drop_index = []
    for idx, row in df.iterrows():
        if math.isnan(row[""][""][""]["Fundamental [cm-1]"]):
            drop_index.append(idx)
            continue
        if not math.isnan(row[""][""][""]["#"]):
            mol_no, mol_name, brutto, _, _, owner, *_ = row
        else:
            row[[
                ["", "", "", "#"],
                ["", "", "", "Molecule"],
                ["", "", "", 'Brutto'],
                ["", "", "", "Who calculated"],
            ]] = mol_no, mol_name, brutto, owner
        df.loc[idx] = row
    df.drop(
        labels=drop_index,
        axis=0,
        inplace=True,
    )

In [12]:
assign_missing_properties(flat_index_data)
flat_index_data

Method                                                                  \
Dispersion                                                               
Basis                                                                    
Property       #             Molecule Brutto Fundamental [cm-1] Source   
0            1.0     Hydrogen cyanide    HCN              712.0    NaN   
1            1.0     Hydrogen cyanide    HCN             2089.0    NaN   
2            1.0     Hydrogen cyanide    HCN             3312.0    NaN   
4            2.0  hydrogen isocyanide    HNC              477.0    NaN   
5            2.0  hydrogen isocyanide    HNC             2029.0    NaN   
..           ...                  ...    ...                ...    ...   
213         54.0      Cyanogen iodide    ICN              486.0    NaN   
214         54.0      Cyanogen iodide    ICN             2188.0    NaN   
216         55.0     Xenon difluoride   XeF2              213.0    NaN   
217         55.0     Xenon difluoride   XeF2              517.0    NaN   
218         55.0     Xenon difluoride   XeF2              560.0    NaN   

Method                        BLYP                                 ...  \
Dispersion                  D3(BJ)                                 ...   
Basis                        6-31G            def2-SVP             ...   
Property   Who calculated ν [cm-1] Exp./Calc. ν [cm-1] Exp./Calc.  ...   
0               Nikolenko      NaN        NaN      NaN        NaN  ...   
1               Nikolenko      NaN        NaN      NaN        NaN  ...   
2               Nikolenko      NaN        NaN      NaN        NaN  ...   
4               Nikolenko      NaN        NaN      NaN        NaN  ...   
5               Nikolenko      NaN        NaN      NaN        NaN  ...   
..                    ...      ...        ...      ...        ...  ...   
213             Nikolenko      NaN        NaN      NaN        NaN  ...   
214             Nikolenko      NaN        NaN      NaN        NaN  ...   
216             Nikolenko      NaN        NaN      NaN        NaN  ...   
217             Nikolenko      NaN        NaN      NaN        NaN  ...   
218             Nikolenko      NaN        NaN      NaN        NaN  ...   

Method           PBE0                         HF-3c             PBEh-3c  \
Dispersion         D4                                                     
Basis        def2-SVP def2-TZVP                                           
Property   Exp./Calc.  ν [cm-1] Exp./Calc. ν [cm-1] Exp./Calc. ν [cm-1]   
0                 NaN       NaN        NaN      NaN        NaN      NaN   
1                 NaN       NaN        NaN      NaN        NaN      NaN   
2                 NaN       NaN        NaN      NaN        NaN      NaN   
4                 NaN       NaN        NaN      NaN        NaN      NaN   
5                 NaN       NaN        NaN      NaN        NaN      NaN   
..                ...       ...        ...      ...        ...      ...   
213               NaN       NaN        NaN      NaN        NaN      NaN   
214               NaN       NaN        NaN      NaN        NaN      NaN   
216               NaN       NaN        NaN      NaN        NaN      NaN   
217               NaN       NaN        NaN      NaN        NaN      NaN   
218               NaN       NaN        NaN      NaN        NaN      NaN   

Method                r2SCAN-3c                     
Dispersion                                          
Basis                                               
Property   Exp./Calc.  ν [cm-1] Exp./Calc. N atoms  
0                 NaN       NaN        NaN       3  
1                 NaN       NaN        NaN       3  
2                 NaN       NaN        NaN       3  
4                 NaN       NaN        NaN       3  
5                 NaN       NaN        NaN       3  
..                ...       ...        ...     ...  
213               NaN       NaN        NaN       3  
214               NaN       NaN        NaN       3  
216             

# Собираем функции

In [20]:
def clean_multiindex(multiindex: tuple) -> tuple:
    return tuple(
        index if not index.startswith("Unnamed: ") else ""
        for index in multiindex
    )

def assign_missing_properties(df):
    # current context
    mol_no, mol_name, brutto, owner = [None] * 4
    drop_index = []
    for idx, row in df.iterrows():
        if math.isnan(row[""][""][""]["Fundamental [cm-1]"]):
            drop_index.append(idx)
            continue
        if not math.isnan(row[""][""][""]["#"]):
            mol_no, mol_name, brutto, _, _, owner, *_ = row
        else:
            row[[
                ["", "", "", "#"],
                ["", "", "", "Molecule"],
                ["", "", "", 'Brutto'],
                ["", "", "", "Who calculated"],
            ]] = mol_no, mol_name, brutto, owner
        df.loc[idx] = row
    df.drop(
        labels=drop_index,
        axis=0,
        inplace=True,
    )

def process_excel_sheet(path: str, n_atoms: int) -> pd.DataFrame:
    raw_data = pd.read_excel(
        path,
        sheet_name=f"{n_atoms}atoms",
        header=(0, 1, 2, 3),
    )

    # Drop empy column and add n_atoms
    raw_data.drop(raw_data.columns[6], axis=1, inplace=True)
    raw_data[("", "", "", "N atoms")] = n_atoms

    # Properly rename index
    cleaned_multiindex = pd.MultiIndex.from_tuples(
        [
            (clean_multiindex(idx))
            for idx in raw_data.columns
        ],
        names=('Method', 'Dispersion', 'Basis', 'Property')
    )

    clean_index_data = pd.DataFrame(columns=cleaned_multiindex)
    for src_col, dst_col in zip(raw_data.columns, clean_index_data.columns):
        clean_index_data[dst_col] = raw_data[src_col]

    # Add mol_no, brutto and owner to all rows
    assign_missing_properties(clean_index_data)
    return clean_index_data

In [31]:
process_excel_sheet(
    EXCEL_PATH,
    12,
).head()

Method                                                            \
Dispersion                                                         
Basis                                                              
Property      #  Molecule       Brutto Fundamental [cm-1] Source   
0           1.0  1-Butene  CH2CHCH2CH3              237.0    NaN   
1           1.0  1-Butene  CH2CHCH2CH3              282.0    NaN   
2           1.0  1-Butene  CH2CHCH2CH3              320.0    NaN   
3           1.0  1-Butene  CH2CHCH2CH3              437.0    NaN   
4           1.0  1-Butene  CH2CHCH2CH3              623.0    NaN   

Method                        BLYP                                 ...  \
Dispersion                  D3(BJ)                                 ...   
Basis                        6-31G            def2-SVP             ...   
Property   Who calculated ν [cm-1] Exp./Calc. ν [cm-1] Exp./Calc.  ...   
0                     NaN      NaN        NaN      NaN        NaN  ...   
1                     NaN      NaN        NaN      NaN        NaN  ...   
2                     NaN      NaN        NaN      NaN        NaN  ...   
3                     NaN      NaN        NaN      NaN        NaN  ...   
4                     NaN      NaN        NaN      NaN        NaN  ...   

Method           PBE0                         HF-3c             PBEh-3c  \
Dispersion         D4                                                     
Basis        def2-SVP def2-TZVP                                           
Property   Exp./Calc.  ν [cm-1] Exp./Calc. ν [cm-1] Exp./Calc. ν [cm-1]   
0                 NaN       NaN        NaN      NaN        NaN      NaN   
1                 NaN       NaN        NaN      NaN        NaN      NaN   
2                 NaN       NaN        NaN      NaN        NaN      NaN   
3                 NaN       NaN        NaN      NaN        NaN      NaN   
4                 NaN       NaN        NaN      NaN        NaN      NaN   

Method                r2SCAN-3c                     
Dispersion                                          
Basis                                               
Property   Exp./Calc.  ν [cm-1] Exp./Calc. N atoms  
0                 NaN       NaN        NaN      12  
1                 NaN       NaN        NaN      12  
2                 NaN       NaN        NaN      12  
3                 NaN       NaN        NaN      12  
4                 NaN       NaN        NaN      12  

[5 rows x 61 columns]

In [28]:
all_data = pd.concat(
    process_excel_sheet(
        EXCEL_PATH,
        n_at,
    )
    for n_at in range(2, 13)
)

In [30]:
all_data.head(5)

Method                                                             \
Dispersion                                                          
Basis                                                               
Property      #         Molecule Brutto Fundamental [cm-1] Source   
0           1.0         Hydrogen     H2             4161.0    NaN   
1           2.0  Lithium Hydride    LiH             1360.0    NaN   
2           3.0         Dicarbon     C2             1827.0    NaN   
3           4.0    Boron nitride     BN             1490.0    NaN   
4           5.0         Nitrogen     N2             2330.0    NaN   

Method                          BLYP                                 ...  \
Dispersion                    D3(BJ)                                 ...   
Basis                          6-31G            def2-SVP             ...   
Property     Who calculated ν [cm-1] Exp./Calc. ν [cm-1] Exp./Calc.  ...   
0           Nikita Matsokin      NaN        NaN      NaN        NaN  ...   
1                 Nikolenko      NaN        NaN      NaN        NaN  ...   
2                 Nikolenko      NaN        NaN      NaN        NaN  ...   
3                 Nikolenko      NaN        NaN      NaN        NaN  ...   
4           Nikita Matsokin      NaN        NaN      NaN        NaN  ...   

Method          PBE0               HF-3c             PBEh-3c             \
Dispersion        D4                                                      
Basis      def2-TZVP                                                      
Property    ν [cm-1] Exp./Calc. ν [cm-1] Exp./Calc. ν [cm-1] Exp./Calc.   
0                NaN        NaN      NaN        NaN      NaN        NaN   
1                NaN        NaN      NaN        NaN      NaN        NaN   
2                NaN        NaN      NaN        NaN      NaN        NaN   
3                NaN        NaN      NaN        NaN      NaN        NaN   
4                NaN        NaN      NaN        NaN      NaN        NaN   

Method     r2SCAN-3c                         BLYP  
Dispersion                                 D3(BJ)  
Basis                                       6-31G  
Property    ν [cm-1] Exp./Calc. N atoms Exp./Calc  
0                NaN        NaN       2       NaN  
1                NaN        NaN       2       NaN  
2                NaN        NaN       2       NaN  
3                NaN        NaN       2       NaN  
4                NaN        NaN       2       NaN  

[5 rows x 62 columns]

In [32]:
a = 1, 2, 3

In [35]:
a[1:3]

(2, 3)